**Libraries**

In [1]:
import os
import numpy as np
import pandas as pd
import scipy as sp
import warnings
from statistics import mean

In [2]:
from sklearn.metrics.pairwise import cosine_similarity #ML Model

In [3]:
pd.options.display.max_columns

20

In [4]:
warnings.filterwarnings("always")
warnings.filterwarnings("ignore")

**Preprocessing of Data**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
anime_df = pd.read_csv('/content/drive/MyDrive/anime.csv')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [7]:
rating_df = pd.read_csv('/content/drive/MyDrive/rating.csv' ,  on_bad_lines='skip') #on_bad_lines = 'skip' :- this will cause the offending lines to be skipped.
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


**Data Shape**

In [8]:
print(f"anime set (row, col): {rating_df.shape}")

anime set (row, col): (7813737, 3)


In [9]:
print(f"anime set (row, col): {anime_df.shape}")

anime set (row, col): (12294, 7)


In [10]:
print("Anime:- \n")
print(anime_df.info())
print("\n" , "*"*50 , "\n Rating :- \n" )
print(rating_df.info())

Anime:- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
None

 ************************************************** 
 Rating :- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
None


**Handling Missing values**

In [11]:
print("Anime_id missing values(%) \n")
print(round(anime_df.isnull().sum().sort_values(ascending=False)/len(anime_df.index) , 4)*100)
print("rating missing values(%) \n")
print(round(rating_df.isnull().sum().sort_values(ascending=False)/len(rating_df.index) , 4)*100)

Anime_id missing values(%) 

rating      1.87
genre       0.50
type        0.20
anime_id    0.00
name        0.00
episodes    0.00
members     0.00
dtype: float64
rating missing values(%) 

user_id     0.0
anime_id    0.0
rating      0.0
dtype: float64


In [12]:
print(anime_df['type'].mode())
print(anime_df['genre'].mode())

0    TV
Name: type, dtype: object
0    Hentai
Name: genre, dtype: object


In [13]:
round(anime_df['rating'].sum()/len(anime_df['rating'].index),1)

6.4

deleting anime with rating 0


In [14]:
anime_df = anime_df[~np.isnan(anime_df['rating'])]
anime_df.isnull().sum()

anime_id     0
name         0
genre       47
type         0
episodes     0
rating       0
members      0
dtype: int64

filling mode value in genre and type

In [15]:
anime_df['genre'] = anime_df['genre'].fillna(
    anime_df['genre'].dropna().mode().values[0]
)

anime_df['type'] = anime_df['type'].fillna(
    anime_df['type'].dropna().mode().values[0]
)

anime_df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

filling NAN in rating in case of -1


In [16]:
rating_df['rating'] = rating_df['rating'].apply(lambda x: np.nan if x==-1 else x)
rating_df.head(20)

,user_id,anime_id,rating
0,1,20,NaN
1,1,24,NaN
2,1,79,NaN
3,1,226,NaN
4,1,241,NaN
5,1,355,NaN
6,1,356,NaN
7,1,442,NaN
8,1,487,NaN
9,1,846,NaN


In [17]:
#step 1
anime_df = anime_df[anime_df['type']=='TV']

#step 2
rated_anime = rating_df.merge(anime_df, left_on = 'anime_id', right_on = 'anime_id', suffixes= ['_user', ''])

#step 3
rated_anime =rated_anime[['user_id', 'name', 'rating']]

In [18]:
pivot = rated_anime.pivot_table(index=['user_id'], columns=['name'], values='rating')
pivot.head()

name,.hack//Roots,.hack//Sign,.hack//Tasogare no Udewa Densetsu,0-sen Hayato,009-1,07-Ghost,11eyes,12-sai.: Chicchana Mune no Tokimeki,12-sai.: Chicchana Mune no Tokimeki 2nd Season,2020 Nyeon Ujuui Wonder Kiddy,...,Zukkoke Knight: Don De La Mancha,Zumomo to Nupepe,ef: A Tale of Melodies.,ef: A Tale of Memories.,gdgd Fairies,gdgd Fairies 2,iDOLM@STER Xenoglossia,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,6.49,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.11,NaN


In [19]:
pivot_n = pivot.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)

pivot_n.fillna(0, inplace=True)

pivot_n = pivot_n.T

pivot_n = pivot_n.loc[:, (pivot_n != 0).any(axis=0)]

piv_sparse = sp.sparse.csr_matrix(pivot_n.values)

In [20]:
anime_similarity = cosine_similarity(piv_sparse)

ani_sim_df = pd.DataFrame(anime_similarity, index = pivot_n.index, columns = pivot_n.index)

In [21]:
def anime_recommendation(ani_name):    
    number = 1

    print('Recommended because you watched {}:\n'.format(ani_name))

    for anime in ani_sim_df.sort_values(by = ani_name, ascending = False).index[1:11]:
        print(f'#{number}: {anime}, {round(ani_sim_df[anime][ani_name]*100,2)}% match')
        
        number +=1

In [22]:
anime_recommendation('Death Note')

Recommended because you watched Death Note:

#1: Code Geass: Hangyaku no Lelouch, 54.79% match
#2: Code Geass: Hangyaku no Lelouch R2, 51.51% match
#3: Shingeki no Kyojin, 48.06% match
#4: Fullmetal Alchemist, 47.97% match
#5: Fullmetal Alchemist: Brotherhood, 46.65% match
#6: Angel Beats!, 44.2% match
#7: Toradora!, 43.14% match
#8: Durarara!!, 41.35% match
#9: Steins;Gate, 41.09% match
#10: Clannad, 40.67% match
